In [4]:
conda install openai

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
| DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/osx-64/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/noarch/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/osx-64/current_repodata.json HTTP/1.1" 304 0
\ DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/noarch/current_repodata.json HTTP/1.1" 304 0
done
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [13]:
import openai
import json
# load and set out key
openai.api_key = open('key.txt','r').read().strip('\n')

# Harnessing Function Calling with OpenAI API for GPT-4 and GPT-3.5

Explore this narrative through the lens of a notebook: link

OpenAI has unveiled a game-changing feature for their models via the API, dubbed "Function Calling." This innovation aims to significantly simplify the process of deriving structured, deterministic information from an inherently unstructured and non-deterministic behemoth like GPT-4.

The endeavor of molding and extracting deterministic outputs from a language model has historically been a Herculean task, sparking a plethora of research. The conventional tactic has been a relentless trial and error with various pre-prompts and few-shot learning examples until stumbling upon a somewhat working solution. Although feasible, the outcome was often cumbersome and lacked reliability. Fast forward to now, the function calling feature paves the way for crafting robust programs, essentially infusing intelligence into your applications.

Picture a scenario where you wish to smartly manage a myriad of input types, including queries like: "What's the weather like in Boston?"

The challenge, when faced with such natural language input for GPT-4, unfolds as follows:

Discerning if the user is in pursuit of weather information.
Upon affirmation, extracting the specified location from their input.
For instance, if the user greets with "Hello, how are you today?", there's no need to trigger the function or attempt to extract a location.

However, when the user inquires, "What's the weather like in Boston?", it's imperative to recognize the intent to fetch weather details and extract the location "Boston" from the query.

In the bygone era, you might have channeled this input to the OpenAI API for GPT-4 in the following manner:

In [16]:
completion = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [{"role":"user","content":"Waht's the weather like in Cardiff UK?"}]
)

And then you'd access the response via:

In [17]:
reply_content = completion.choices[0].message.content
print(reply_content)

I'm sorry, but as an AI language model, I don't have access to real-time data. However, Cardiff generally has a maritime climate with mild winters and relatively cool summers. It often experiences frequent rainfall throughout the year. To get the most accurate and up-to-date weather information for Cardiff, I recommend checking a reliable weather website or using a weather app.


As you can see, this isn't quite what we would want to happen in this scenario! While GPT-4 may not currently be able to access the internet for us, we could conceivably do this ourselves, but we still would need to identify the intent, as well as the particular desired location. Imagine we have a function like:

In [19]:
def get_current_weather(location,unit="fahrenheit"):
    #get the current weather in a given location
    weather_info = {
        "location":location,
        "unit":unit,
        "temperature": 20,
        "humidity": 80,
        "wind": 10,
        "rain": 0,
        "snow": 0,
        "forecast":["sunny","windy"],

    }
    return weather_info

This is just a placeholder type of function to show how this all ties together, but it could be anything here. Extracting the intent and location could be done with a preprompt, and this is sort of how OpenAI is doing it through the API, but the model has been trained for us with a particular structure, so we can use this to save a lot of R&D time to find the "best prompt" to get it done.

To do this, we want to make sure that we're using version `gpt-4-0613` or later. Then, we can pass a new functions parameter to the `ChatCompletion` call like:

In [21]:
model = "gpt-4-0613"
messages = [{"role":"user","content":"What's the weather like in Cardiff UK?"}]
functions = [
        {
         "name": "get_current_weather",
         "description":"Get the current weather in given location",
         "parameters":{
             "type":"object",
             "properties":{
                 "location":{
                     "type":"string",
                     "description":"the city and state, e.g. San Francisco, CA",
                 },
                 "unit":{
                     "type":"string",
                     "description":"the unit of temperature",
                     "enum":["celsius","fahrenheit"]},
             },
             "required":["location"],
         },
         }
    ]

In [22]:
completion = openai.ChatCompletion.create(
    model = model,
    messages = messages,
    functions = functions,
    function_call= "auto",
)

The first thing to note is the `function_call="auto"`, part. This will let GPT-4 determine if it should seek to fulfill the function parameters. You can also set it to none to force no function to be detected, and finally you can set it to seek parameters for a specific function by doing something like `function_call={"name": "get_current_weather"}`. There are many instances where it could be hard for GPT-4 to determine if a function should be run, so being able to force it to run if you know it should be is actually very powerful, which I'll show soon.

Beyond this, we name and describe the function, then describe the parameters that we'd hope to pass to this function. GPT-4 is relying on this description to help identify what it is you want, so try to be as clear as possible here. The API is going to return to you a json structured object, and this is how you structure your function description, which affords you quite a bit of flexibility in how you describe/structure this functionality.

Okay let's see how GPT-4 responds to our new prompt: